In [1]:
pwd

'/Users/kelly/metis_v3/Project_5'

In [66]:
book_df = pd.read_csv('books.csv')
ratings_df = pd.read_csv('ratings.csv')
booktags_df = pd.read_csv('booktags.csv')
tags_df = pd.read_csv('tags.csv')

In [8]:
import numpy as np
import pandas as pd

In [17]:
from sklearn.neighbors import NearestNeighbors

In [13]:
from pymongo import MongoClient
client = MongoClient()
db = client.goodreads

In [4]:
import pickle

with open('U.pickle','rb') as read_file:
    U = pickle.load(read_file)

In [26]:
with open('book_vecs_df_drop_na.pickle','rb') as read_file:
    book_vecs_df = pickle.load(read_file)

In [27]:
with open('book_vecs_drop_title.pickle','rb') as read_file:
    book_vecs = pickle.load(read_file)

# Import these functions when conver to .py files!!!!!!

**Collaborative Filtering**

In [315]:
def find_book_rating(ID):
    for book in db.books.find({'book_id':ID}):
        print (book['average_rating'])

def find_book_title(ID):
    for book in db.books.find({'book_id':ID}):
        print(book['original_title'])
        return (book['original_title'])

In [331]:
def find_book_title_noprint(ID):
    for book in db.books.find({'book_id':ID}):
        #print(book['original_title'])
        return (book['original_title'])

In [313]:
def get_recommendations_for_user(userID, U, VT, num_recom):
    recs = []
    for item in range(VT.T.shape[0]):
        recs.append([item+1,np.dot(U[userID-1],VT.T[item])])
    final_rec = [(i[0],i[1]) for i in sorted(recs,key=lambda x: x[1],reverse=True)]
    return final_rec[:num_recom]

**Content-Based Filtering**

In [20]:
def get_title_index(book_title):
    try:
        filtered_vecs = book_vecs_df[book_vecs_df.Title == book_title]
        vec_index = filtered_vecs.index[0]
    except:
        print("Book currently checked out, please pick another")
    return vec_index

In [454]:
def get_reading_list(last_book_read):
    
    vec_index = get_title_index(last_book_read)
    
    nn = NearestNeighbors(n_neighbors=1000, metric='cosine', algorithm='brute')
    nn.fit(book_vecs)
    book_recs = nn.kneighbors(np.array([book_vecs.iloc[vec_index]]))
    
    reading_list = book_recs[1][0]
    
    reading_list_recs = []
    
    '''
    from collections import defaultdict

    rec_dict = defaultdict(list) #initializes default dictionary for book recommendations

    
    for ix, book_ix in enumerate(reading_list):
        #print (ix, book_ix)
        rec_dict[last_book_read].append(book_vecs_df.iloc[book_ix].Title)
        
    return rec_dict   
    '''
    
    for book in reading_list:
        if book != '':
            reading_list_recs.append(book_vecs_df.iloc[book].Title)
        else:
            continue
        
    return reading_list_recs

In [494]:
USERID=1
num_recom = 10
for item in get_recommendations_for_user(USERID,U,VT,num_recom):
    find_book_title(item[0])
    find_book_rating(item[0])

The Hunger Games
4.34
Harry Potter and the Philosopher's Stone
4.44
Twilight
3.57
To Kill a Mockingbird
4.25
The Great Gatsby
3.89
The Fault in Our Stars
4.26
The Hobbit or There and Back Again
4.25
The Catcher in the Rye
3.79
Angels & Demons
3.85
Pride and Prejudice
4.24


In [631]:
get_reading_list("The Hunger Games")

['Vespers Rising',
 'Beauty from Pain',
 'The Gathering Storm',
 'Labor Day',
 'The Help',
 'Beautiful Secret',
 'The Baller: A Down and Dirty Football Novel',
 'Clockwork Princess',
 'The Lions of Little Rock',
 "You're the one that I want",
 'Sweetheart',
 'Amulet: Book Three: The Cloud Searchers',
 'The Other Side of Dawn',
 'Come Away with Me',
 'Free Four: Tobias Tells the Divergent Story',
 'Skeleton Key',
 'Release Me (Stark Trilogy, #1)',
 '',
 'Fame',
 'The Hunger Games',
 'Playing for Keeps',
 'The Creative License: Giving Yourself Permission to Be The Artist You Truly Are',
 'In Too Deep (The 39 Clues, Book 6)',
 'Civil War: A Marvel Comics Event',
 'Wideacre',
 'The Prefect',
 'Dial L for Loser (The Clique, #6)',
 '高校デビュー 1',
 '',
 'Hogfather',
 'A Streetcar Named Desire',
 'Dear John',
 'The Sight',
 'The Third Wheel',
 'Ἀντιγόνη',
 'Vampireville (Vampire Kisses, #3)',
 'Pandora Hearts 1',
 '',
 'The Queen of the Tearling',
 'Into the Wild',
 'Need',
 'The Iron Daughter',


In [30]:
get_reading_list('The Hunger Games')

defaultdict(list,
            {'The Hunger Games': ['Wideacre',
              'Free Four: Tobias Tells the Divergent Story',
              'The Baller: A Down and Dirty Football Novel',
              '',
              'Playing for Keeps',
              'Sweetheart',
              'Civil War: A Marvel Comics Event',
              'In Too Deep (The 39 Clues, Book 6)',
              'The Help',
              'The Other Side of Dawn']})

**Hybrid Filtering**

In [32]:
'''
Combine the two methods above.  Recommend books by user ratings 
and then filter down recommendation listing based on related content.
'''

'\nCombine the two methods above.  Recommend books by user ratings \nand then filter down recommendation listing based on related content.\n'

In [492]:
def find_book_id(input_title):
    for book in db.books.find({'original_title':input_title}):
        print (book['book_id'])

In [509]:
def find_goodreads_id(input_title):
    for book in db.books.find({'original_title':input_title}):
        print (book['goodreads_book_id'])
        return (book['goodreads_book_id'])

In [544]:
def find_goodreads_id_np(input_title):
    for book in db.books.find({'original_title':input_title}):
        #print (book['goodreads_book_id'])
        return (book['goodreads_book_id'])

In [495]:
#input_title = "Winnie-the-Pooh"
input_title = "Freakonomics: A Rogue Economist Explores the Hidden Side of Everything"
find_book_id(input_title)

92


In [47]:
db.books.find_one()

{'_id': ObjectId('5b88855b21c3f05eeaf945eb'),
 'book_id': 1,
 'goodreads_book_id': 2767052,
 'best_book_id': 2767052,
 'work_id': 2792775,
 'books_count': 272,
 'isbn': 439023483,
 'isbn13': 9780439023480.0,
 'authors': 'Suzanne Collins',
 'original_publication_year': 2008.0,
 'original_title': 'The Hunger Games',
 'title': 'The Hunger Games (The Hunger Games, #1)',
 'language_code': 'eng',
 'average_rating': 4.34,
 'ratings_count': 4780653,
 'work_ratings_count': 4942365,
 'work_text_reviews_count': 155254,
 'ratings_1': 66715,
 'ratings_2': 127936,
 'ratings_3': 560092,
 'ratings_4': 1481305,
 'ratings_5': 2706317,
 'image_url': 'https://images.gr-assets.com/books/1447303603m/2767052.jpg',
 'small_image_url': 'https://images.gr-assets.com/books/1447303603s/2767052.jpg'}

In [58]:
books_by_user = ratings_df.groupby(by = 'user_id')

In [70]:
ratings_df[ratings_df.user_id == 1][ratings_df[ratings_df.user_id == 1].rating == 5]

,user_id,book_id,rating
0,1,258,5
78,1,1796,5
230489,1,11,5
230490,1,1644,5
230492,1,136,5
230495,1,35,5
230502,1,4,5
230505,1,1521,5
230506,1,70,5
230516,1,2002,5


In [316]:
USERID=1
num_recom = 100
for item in get_recommendations_for_user(USERID,U,VT,num_recom):
    find_book_title(item[0])
    find_book_rating(item[0])

The Hunger Games
4.34
Harry Potter and the Philosopher's Stone
4.44
Twilight
3.57
To Kill a Mockingbird
4.25
The Great Gatsby
3.89
The Fault in Our Stars
4.26
The Hobbit or There and Back Again
4.25
The Catcher in the Rye
3.79
Angels & Demons
3.85
Pride and Prejudice
4.24
The Kite Runner
4.26
Divergent
4.24
Nineteen Eighty-Four
4.14
Animal Farm: A Fairy Story
3.87
Het Achterhuis: Dagboekbrieven 14 juni 1942 - 1 augustus 1944
4.1
Män som hatar kvinnor
4.11
Catching Fire
4.3
Harry Potter and the Prisoner of Azkaban
4.53
The Fellowship of the Ring
4.34
Mockingjay
4.03
Harry Potter and the Order of the Phoenix
4.46
The Lovely Bones
3.77
Harry Potter and the Chamber of Secrets
4.37
Harry Potter and the Goblet of Fire
4.53
Harry Potter and the Deathly Hallows
4.61
The Da Vinci Code
3.79
Harry Potter and the Half-Blood Prince
4.54
Lord of the Flies
3.64
An Excellent conceited Tragedie of Romeo and Juliet
3.73
Gone Girl
4.03
The Help
4.45
Of Mice and Men
3.84
Memoirs of a Geisha
4.08
Fifty Sha

In [629]:
#People who rated Freakonomics a 5
#ratings_df[ratings_df.book_id == 92][ratings_df[ratings_df.book_id == 92].rating == 5]

In [630]:
#People who rated Winnie the Pooh a 5
#ratings_df[ratings_df.book_id == 444][ratings_df[ratings_df.book_id == 444].rating == 5]

In [279]:
original_rec_list = []

In [381]:
def create_rec_list(user_id):
    original_recs = []
    for item in get_recommendations_for_user(user_id, U, VT, 1000):
        original_recs.append(find_book_title_noprint(item[0]))
    return original_recs

In [345]:
create_rec_list(60)

['The Glass Castle',
 'Angels & Demons',
 'A Thousand Splendid Suns',
 'Fifty Shades of Grey',
 'Animal Farm: A Fairy Story',
 'The Notebook',
 'Memoirs of a Geisha',
 'The Picture of Dorian Gray',
 'Harry Potter and the Half-Blood Prince',
 'Het Achterhuis: Dagboekbrieven 14 juni 1942 - 1 augustus 1944',
 'The Girl on the Train',
 'The Fault in Our Stars',
 'The Kite Runner',
 'A Clockwork Orange',
 'Flowers for Algernon',
 'The Perks of Being a Wallflower',
 'The Lost Hero',
 'Speak',
 'If I Stay',
 'Dark Places',
 '',
 'The Bell Jar',
 'A Discovery of Witches',
 "The Titan's Curse",
 'The Five People You Meet in Heaven',
 'Kiss the Girls',
 'The Graveyard Book',
 'Freedom',
 "A People's History of the United States: 1492 to Present",
 'A Clash of Kings',
 'Uglies',
 'Harry Potter and the Cursed Child, Parts One and Two',
 'The Night Circus',
 'The Dead Zone',
 'Choke',
 'The Phantom Tollbooth',
 '',
 'American Psycho',
 'In Cold Blood',
 'City of Heavenly Fire',
 'The Reptile Room',

In [335]:
book_title = "Freakonomics: A Rogue Economist Explores the Hidden Side of Everything"
get_reading_list(book_title)

defaultdict(list,
            {'Freakonomics: A Rogue Economist Explores the Hidden Side of Everything': ["Galileo's Daughter",
              '',
              'The Cake Mix Doctor',
              'Sons and Lovers',
              'New Spring',
              'Chasing Fire',
              'Sea Glass',
              'The Body Farm',
              'The Empty Chair',
              'Five Quarters of the Orange',
              'Animal Dreams',
              'Washington Square',
              'The Devil Wears Prada',
              'The Restaurant at the End of the Universe',
              'Club Dead',
              'Don Quijote de La Mancha',
              'Pebble in the Sky',
              'Down Under',
              'Beautiful Day',
              'Port Mortuary',
              'Memorial do Convento',
              'Unnatural Exposure',
              'The Sands of Time',
              'A History of Western Philosophy And Its Connection with Political and Social Circumstances from the Earliest

In [474]:
#find books user has previously rated

def previously_read(user_num):

    userread_df = ratings_df[ratings_df.user_id == user_num].join(
    book_df, how = 'left', lsuffix = ' ', on = 'book_id')[['user_id','book_id','original_title']]
    
    previously_readbooks = list(userread_df.original_title)
    
    return previously_readbooks

In [262]:
#find books user has rated a 5

def find_favorites(user_num):

    userread_df = ratings_df[ratings_df.user_id == user_num][ratings_df[ratings_df.user_id == user_num].rating == 5].join(
    book_df, how = 'left', lsuffix = ' ', on = 'book_id')[['user_id','book_id','original_title']]
    
    fav_books = list(userread_df.original_title)
    
    return fav_books

In [550]:
def find_bookurl(book_title):
    goodreads_book_id = find_goodreads_id_np(book_title)
    return 'https://www.goodreads.com/book/show/'+str(goodreads_book_id)

In [627]:
def hybrid_filtering(user_id):
    
    last_book_read = np.random.choice(find_favorites(user_id)) #picks favorite book from user's previously read books
    print('Last Book Read:', last_book_read)
    
    already_read_list = previously_read(user_id)   
    original_list = create_rec_list(user_id) #takes in user id and returns original rec list
    reading_list = get_reading_list(last_book_read) #returns reading list
    
    book_recs = []
    
    for book in original_list:
        if book in reading_list:
            if book in reading_list == '':
                pass
            elif book not in already_read_list:
                book_recs.append(book)
    
    from collections import defaultdict
    url_dict = defaultdict(list)
    
    #url_dict = {}
    for ix, rec in enumerate(book_recs[:10]):
        book_link = find_bookurl(rec)
        url_dict[ix] = book_link
    
    print('Book Recommendations:', book_recs[:10])
    #print(url_dict)

In [628]:
hybrid_filtering(600)

Last Book Read: Mockingjay
Book Recommendations: ['Last Sacrifice', '1st To Die', 'Her Fearful Symmetry', 'Rivers of London', 'Stone Cold', '', '', 'The Guard', 'Extremely Loud and Incredibly Close', 'The Way of Kings']
